In [1]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd 
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service
import os

#### Web Page Elements 
Videos_List : style-scope ytd-rich-item-renderer

title --  //*[@id="video-title-link"]

views -- //*[@id="metadata-line"]/span[1]

day_posted -- //*[@id="metadata-line"]/span[2]
video_link -- //*[@id="thumbnail"]


In [26]:
PATH = r'chromedriver.exe'
service = Service(executable_path=PATH)
wd = webdriver.Chrome(service=service)
wd.get(url)
videos = wd.find_elements(By.CLASS_NAME, "style-scope ytd-rich-item-renderer")

for video in videos:
    link = video.find_element(By.CLASS_NAME,"yt-simple-endpoint inline-block style-scope ytd-thumbnail")
    print(link.get_attribute('href'))


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".yt-simple-endpoint inline-block style-scope ytd-thumbnail"}
  (Session info: chrome=116.0.5845.110); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7CEB252A2+57122]
	(No symbol) [0x00007FF7CEA9EA92]
	(No symbol) [0x00007FF7CE96E3AB]
	(No symbol) [0x00007FF7CE9A7D3E]
	(No symbol) [0x00007FF7CE9A7E2C]
	(No symbol) [0x00007FF7CE99EC3C]
	(No symbol) [0x00007FF7CE9C701F]
	(No symbol) [0x00007FF7CE99EB76]
	(No symbol) [0x00007FF7CE9C71F0]
	(No symbol) [0x00007FF7CE9DEB82]
	(No symbol) [0x00007FF7CE9C6DB3]
	(No symbol) [0x00007FF7CE99D2B1]
	(No symbol) [0x00007FF7CE99E494]
	GetHandleVerifier [0x00007FF7CEDCEF82+2849794]
	GetHandleVerifier [0x00007FF7CEE21D24+3189156]
	GetHandleVerifier [0x00007FF7CEE1ACAF+3160367]
	GetHandleVerifier [0x00007FF7

## Scrolling Till 50 Links are reached


In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

def scroll_to_end(wd):
    wd.execute_script("window.scrollTo(0, 1000);")
    time.sleep(1)

def fetch_video_details(url, max_videos=50):
    PATH = r'chromedriver.exe'
    service = Service(executable_path=PATH)
    wd = webdriver.Chrome(service=service)
    wd.get(url)
    
    video_details = []
    while len(video_details) < max_videos:
        videos = wd.find_elements(By.CLASS_NAME, "style-scope ytd-rich-item-renderer")
        scroll_to_end(wd)
        
        for video in videos:
            title = video.find_element(By.XPATH, ".//*[@id='video-title-link']").text
            view = video.find_element(By.XPATH, ".//*[@id='metadata-line']/span[1]").text
            date = video.find_element(By.XPATH, ".//*[@id='metadata-line']/span[2]").text
            # duration = video.find_element(By.CLASS_NAME, "style-scope ytd-thumbnail-overlay-time-status-renderer").text
            
            details = {
                'title': title,
                'views': view,
                'date': date
            }
            
            video_details.append(details)
            if len(video_details) >= max_videos:
                break
    
    wd.quit()
    return pd.DataFrame(video_details)

url = "https://www.youtube.com/@krishnaik06/videos"
max_videos = 50
video_details_df = fetch_video_details(url, max_videos)


In [23]:
video_details_df.head()

,title,views,date
0,48 Years Old Transition From Faculty To Data S...,1.7K views,3 hours ago
1,Evidently AI Tutorial-Open Source ML Models Mo...,5.7K views,2 days ago
2,Is Transition From Mechanical Engineering To D...,11K views,5 days ago
3,BentoML Tutorial: Build Production Grade AI Ap...,8.2K views,7 days ago
4,Rohan Successful Transition-Freshers To MLOPS ...,8.3K views,9 days ago
